## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-09-01-04-27 +0000,startribune,Live: Updates from Minnesota girls volleyball ...,https://www.startribune.com/high-school-volley...
1,2025-11-09-01-00-00 +0000,wsj,U.S. elections are sending a consistent messag...,https://www.wsj.com/politics/elections/america...
2,2025-11-09-00-51-00 +0000,bbc,"More than 1,400 flights cancelled as US air tr...",https://www.bbc.com/news/articles/cj410k00yw8o...
3,2025-11-09-00-47-19 +0000,nypost,Air Force families at Florida base ordered to ...,https://nypost.com/2025/11/08/us-news/air-forc...
4,2025-11-09-00-46-40 +0000,latimes,Widespread power outage temporarily impacts mo...,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2447/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
144,trump,30
76,snap,9
90,shutdown,8
91,flight,8
15,government,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
185,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...,73
176,2025-11-08-03-45-03 +0000,nyt,Trump Administration Appeals to Supreme Court ...,https://www.nytimes.com/2025/11/07/us/politics...,67
178,2025-11-08-02-59-00 +0000,wsj,Supreme Court Pauses Order for Trump Administr...,https://www.wsj.com/us-news/law/supreme-court-...,66
70,2025-11-08-17-30-00 +0000,wsj,"Middle-class voters, many of them frustrated w...",https://www.wsj.com/politics/elections/democra...,65
168,2025-11-08-05-16-33 +0000,bbc,Trump gives Hungary one-year exemption from Ru...,https://www.bbc.com/news/articles/cp85dvnkpv4o...,56


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
185,73,2025-11-08-02-00-00 +0000,wsj,Republicans in Washington got a clear takeaway...,https://www.wsj.com/politics/policy/trump-pric...
78,47,2025-11-08-17-17-26 +0000,nypost,New Yorkers flock to food banks as government ...,https://nypost.com/2025/11/08/us-news/trump-ur...
95,37,2025-11-08-14-29-08 +0000,cbc,"UPS, FedEx ground MD-11 air cargo fleets after...",https://www.cbc.ca/news/world/ups-fedex-ground...
52,32,2025-11-08-20-19-55 +0000,nypost,Singer claims transgender woman — who beat ex-...,https://nypost.com/2025/11/08/us-news/transgen...
73,31,2025-11-08-17-25-55 +0000,nypost,Mamdani-backing Hochul insists she doesn’t ‘su...,https://nypost.com/2025/11/08/us-news/mamdani-...
74,31,2025-11-08-17-23-43 +0000,nypost,"Andrew Mountbatten Windsor, Fergie being probe...",https://nypost.com/2025/11/08/world-news/princ...
35,28,2025-11-08-21-59-45 +0000,nypost,"At least 5 dead, hundreds injured after tornad...",https://nypost.com/2025/11/08/world-news/at-le...
176,28,2025-11-08-03-45-03 +0000,nyt,Trump Administration Appeals to Supreme Court ...,https://www.nytimes.com/2025/11/07/us/politics...
111,24,2025-11-08-12-38-25 +0000,nyt,Pfizer to Acquire Obesity Drug Start-Up Metser...,https://www.nytimes.com/2025/11/08/business/me...
54,23,2025-11-08-20-15-55 +0000,nypost,Shooter opens fire at Border Patrol agents in ...,https://nypost.com/2025/11/08/us-news/border-p...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
